In [1]:
!pip install causalml

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import copy
from causalml.dataset import *

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
y, X, treatment, tau, b, e = synthetic_data(mode=2, n=10000, p=8, sigma=1.0)
X_train, X_test, y_train, y_test, treat_train, treat_test= train_test_split(X, y, treatment, test_size=0.33, random_state=0)
train_idx, val_idx = train_test_split(np.arange(len(X_train)), test_size=0.3, random_state=123)
X_val, y_val, treat_val = X_train[val_idx], y_train[val_idx], treat_train[val_idx]
X_train, y_train, treat_train = X_train[train_idx], y_train[train_idx], treat_train[train_idx]

action_probs = np.bincount(treat_train) / len(treat_train)

In [4]:
class X5Dataset(Dataset):
    def __init__(self, X, y, treat):
        self.X = torch.from_numpy(X).type(torch.FloatTensor)
        self.y = torch.from_numpy(y).type(torch.LongTensor)
        self.treat = treat
  
    def __getitem__(self, id):
        return self.X[id], self.y[id], self.treat[id]
  
    def __len__(self):
        return len(self.X)

def collate(batch):
    X, y, treat = zip(*batch)
    return X, y, treat

In [5]:
class PolicyGradient(nn.Module):
    def __init__(self, n_action, n_features, hidden_dims):
        super(PolicyGradient, self).__init__()

        def get_dense_block(in_dim, out_dim):
            return nn.Sequential(nn.Linear(in_dim, out_dim), nn.BatchNorm1d(out_dim), nn.Tanh())

        blocks = [get_dense_block(n_features, hidden_dims[0])]
        for i in range(len(hidden_dims) - 1):
            blocks.append(get_dense_block(hidden_dims[i], hidden_dims[i + 1]))
        self.net = nn.Sequential(*blocks,
                                nn.Linear(hidden_dims[-1], n_action))

    def forward(self, features):
        return self.net(features)

In [6]:
class Trainer:
    def __init__(self, model, action_probs, train_dataset, val_dataset, test_dataset, train_config, device, n_actions=2):
        self.model = model
        self.model.to(device)
        self.train_loader = DataLoader(train_dataset, 
                                   batch_size=train_config.batch_size, 
                                   drop_last=True,
                                   shuffle=True)
        self.val_loader = DataLoader(val_dataset,
                                 batch_size=train_config.batch_size)
        self.test_loader = DataLoader(test_dataset,
                                  batch_size=train_config.batch_size)
        self.train_config = train_config
        self.device = device
        self.n_actions = 2
 
        self.action_probs = action_probs

        self.loss = nn.CrossEntropyLoss(reduce=False)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=train_config.lr)

    def _SN_UMG(self, records, n_actions=2):
        """
        records: a sequence of [algorithm_action, actual_action, y]
        """
        response_numerator = 0
        response_denominator = 0
        control_numerator = np.zeros(n_actions)
        control_denominator = np.zeros(n_actions)

        for alg_a, act_a, y in records:
            if alg_a == act_a:
                response_numerator += y / self.action_probs[act_a]
                response_denominator += 1 / self.action_probs[act_a]
            if act_a == 0 and alg_a > 0:
                control_numerator[alg_a] += y / self.action_probs[0]
                control_denominator[alg_a] += 1 / self.action_probs[0]
    
        response = response_numerator / (response_denominator if response_denominator != 0 else 1)
        action_control = control_numerator / np.where(control_denominator == 0, 1, control_denominator)

        lift = response - control_numerator.sum() / (control_denominator.sum() if control_denominator.sum() != 0 else 1)

        return lift, action_control

    def _SN_UMG_GH(self, records, n_action=2):
        n_treat = n_action - 1
        real_prob = self.action_probs
        algo_action_resp = {}
        algo_total_norm = 0.0
        algo_total_base_norm = 0.0
        algo_treat_resp = [0.0] * n_treat
        algo_total_resp = 0.0
        algo_action_base = {}
        algo_treat_base = [0.0] * n_treat
        algo_total_base = 0.0
        algo_treat_lift = [0.0] * n_treat
        algo_total_lift = 0.0
        algo_action_nums = np.array([0.0] * n_action)
        algo_action_norm = np.array([0.0] * n_action)
        algo_action_base_norm = np.array([0.0] * n_action)

        real_action_resp = {}
        real_treat_resp = [0.0] * n_treat
        real_total_resp = 0.0
        real_action_nums = np.array([0.0] * n_action)

        for t in range(n_treat):
            algo_action_resp[t] = np.array([0.0] * n_action)
            algo_action_base[t] = np.array([0.0] * n_action)

            real_action_resp[t] = np.array([0.0] * n_action)

        for algo_action, real_action, resp in records:
            algo_action_nums[algo_action] += 1
            real_action_nums[real_action] += 1

            algo_action_resp[0][algo_action] += resp * \
                (real_action == algo_action) / real_prob[real_action]
            algo_treat_resp[0] += resp * \
                (real_action == algo_action) / real_prob[real_action]
            algo_action_base[0][algo_action] += resp * \
                (real_action == 0) / real_prob[0]
            algo_treat_base[0] += resp * \
                (real_action == 0) / real_prob[0]

            real_action_resp[0][real_action] += resp
            real_treat_resp += resp

            algo_action_norm[algo_action] += (real_action == algo_action) * \
                1.0 / real_prob[real_action]
            algo_total_norm += (real_action == algo_action) / \
                real_prob[real_action]
            algo_action_base_norm[real_action] += (
                real_action == 0) * 1.0 / real_prob[real_action]
            algo_total_base_norm += (real_action == 0) * \
                1.0 / real_prob[real_action]

        numSample = np.sum(algo_action_nums)

        for a in range(n_action):
            algo_action_resp[0][a] /= (algo_action_norm[a]
                                    if algo_action_norm[a] > 0.0 else 1.0)
            algo_action_base[0][a] /= (algo_action_norm[a]
                                    if algo_action_norm[a] != 0.0 else 1.0)
            real_action_resp[0][a] /= (real_action_nums[a]
                                    if real_action_nums[a] != 0.0 else 1.0)
        algo_treat_base[0] /= (algo_total_base_norm if algo_total_base_norm >
                            0.0 else 1.0)
        algo_treat_resp[0] /= (algo_total_norm if algo_total_norm >
                            0.0 else 1.0)
        real_treat_resp[0] /= numSample

        algo_action_prob = algo_action_nums / np.sum(algo_action_nums)
        real_action_prob = real_action_nums / np.sum(real_action_nums)

        for i in range(n_treat):
            for a in range(n_action):
                algo_treat_lift[i] += (algo_action_resp[i][a] -
                                   algo_action_base[i][a]) * algo_action_prob[a]

        for i in range(n_treat):
            algo_total_lift += algo_treat_lift[i]
            algo_total_resp += algo_treat_resp[i]
            algo_total_base += algo_treat_base[i]

            real_total_resp += real_treat_resp[i]

        return algo_total_lift, algo_action_base
      
    def _get_train_metrics(self, set_of_records, n_actions=2):
        lifts = []
        action_controls = []
        for records in set_of_records:
            lift, action_control = self._SN_UMG_GH(records, n_action=n_actions)
            lifts.append(lift)
            action_controls.append(action_control[0])
    
        return lifts, np.mean(action_controls, axis=0)

    def _train_epoch(self):
        epoch_features = []
        epoch_actions = []
        epoch_rewards = []
        epoch_treats = []
        epoch_ys = []

        preds = []
        ys = []
        treats = []

        self.model.eval()
        for X, y, treat in self.train_loader:
            epoch_features.append(X)
            X = X.to(device)

            set_of_records = []
            with torch.no_grad():
                logits = self.model(X)
                actions = torch.argmax(logits, 1)

                actions = actions.cpu().numpy()
                treat = treat.numpy()
                y = y.numpy()

                preds.extend(actions)
                treats.extend(treat)
                ys.extend(y)

                records = list(zip(actions, treat, y))
                set_of_records.append(records)
          
                epoch_ys.append(y)
                epoch_actions.append(actions)
                epoch_treats.append(treat)
            
        lifts, action_control = self._get_train_metrics(set_of_records, self.n_actions)
        avg_lift = np.mean(lifts)

        sn_umg = self._SN_UMG_GH([record for records in set_of_records for record in records], n_action=self.n_actions)[0]

        print(f'Train SN-UMG: {round(sn_umg, 4)}')

        for X, y, treat, actions, lift in zip(epoch_features, epoch_ys, epoch_treats, epoch_actions, lifts):
            rewards = np.zeros(len(X))
            for a in range(self.n_actions):
                a_control = action_control[a]

                marker = ((actions == a) & (actions == treat)).astype(np.int16)
                rewards += marker * ((y - a_control) + lift - avg_lift)
                marker = ((treat == 0) & (actions == a) & (a > 0)).astype(np.int16)
                rewards += marker * (-(y - a_control) + lift - avg_lift)
                epoch_rewards.append(torch.Tensor(rewards))

        self.model.train()
        for X, actions, rewards in zip(epoch_features, epoch_actions, epoch_rewards):
            bs = X.size(0)
            X = X.to(device)
            logits = self.model(X)
            probs = torch.softmax(logits, dim=-1)
            entropy = (-probs * torch.log(probs)).sum(-1).mean()

            minus_log_prob = -torch.log(probs[torch.arange(bs), actions])
            entropy = (torch.log(probs) * probs).mean(1)
            loss = (minus_log_prob * rewards + entropy).mean()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
  
    def validate(self):
        preds = []
        ys = []
        treats = []

        records = []

        self.model.eval()
        with torch.no_grad():
            for X, y, treat in self.val_loader:
                logits = self.model(X)
                probs = torch.softmax(logits, 1)
                actions = torch.argmax(logits, 1).cpu().numpy()

                treat = treat.cpu().numpy()
                y = y.cpu().numpy()

                preds.extend(probs[:, 1].cpu().numpy())
                ys.extend(y)
                treats.extend(treat)

                batch_records = list(zip(actions, treat, y))
                records.extend(batch_records)
    
        sn_umg = self._SN_UMG_GH(records, self.n_actions)[0]

        print(f'Val SN-UMG: {round(sn_umg, 4)}\n')

        return sn_umg
  
    def train(self):
        best_score = 0
        best_params = None

        for epoch in range(self.train_config.epochs):
            print(f'Epoch #{epoch + 1}:')
            self._train_epoch()
            score = self.validate()
            if score > best_score:
                best_score = score
                best_params = copy.deepcopy(self.model.state_dict())
    
        self.model.load_state_dict(best_params)

    def test(self):
        preds = []
        ys = []
        treats = []

        records = []

        self.model.eval()
        with torch.no_grad():
            for X, y, treat in self.test_loader:
                logits = self.model(X)
                probs = torch.softmax(logits, 1)
                actions = torch.argmax(logits, 1).cpu().numpy()

                treat = treat.cpu().numpy()
                y = y.cpu().numpy()

                preds.extend(probs[:, 1].cpu().numpy())
                ys.extend(y)
                treats.extend(treat)

                batch_records = list(zip(actions, treat, y))
                records.extend(batch_records)
    
        sn_umg = self._SN_UMG_GH(records, self.n_actions)[0]

        print(f'\nTest SN-UMG: {round(sn_umg, 4)}')
  

In [7]:
class TrainConfig:
    epochs = 100
    lr = 1e-3
    batch_size = 2000

In [8]:
torch.manual_seed(38)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PolicyGradient(2, 8, [20, 15, 5])
trainer = Trainer(model, 
                  action_probs, 
                  X5Dataset(X_train, y_train, treat_train), 
                  X5Dataset(X_val, y_val, treat_val),
                  X5Dataset(X_test, y_test, treat_test),
                  TrainConfig(), 
                  device)

In [9]:
trainer.train()
trainer.test()

Epoch #1:
Train SN-UMG: 0.5976
Val SN-UMG: 0.6909

Epoch #2:
Train SN-UMG: 0.7096
Val SN-UMG: 0.6889

Epoch #3:
Train SN-UMG: 0.6336
Val SN-UMG: 0.6888

Epoch #4:
Train SN-UMG: 0.7223
Val SN-UMG: 0.6787

Epoch #5:
Train SN-UMG: 0.7192
Val SN-UMG: 0.6747

Epoch #6:
Train SN-UMG: 0.6454
Val SN-UMG: 0.6735

Epoch #7:
Train SN-UMG: 0.6353
Val SN-UMG: 0.6831

Epoch #8:
Train SN-UMG: 0.6781
Val SN-UMG: 0.6878

Epoch #9:
Train SN-UMG: 0.667
Val SN-UMG: 0.6871

Epoch #10:
Train SN-UMG: 0.6985
Val SN-UMG: 0.6885

Epoch #11:
Train SN-UMG: 0.6967
Val SN-UMG: 0.6891

Epoch #12:
Train SN-UMG: 0.6464
Val SN-UMG: 0.696

Epoch #13:
Train SN-UMG: 0.7293
Val SN-UMG: 0.6904

Epoch #14:
Train SN-UMG: 0.7152
Val SN-UMG: 0.6894

Epoch #15:
Train SN-UMG: 0.7446
Val SN-UMG: 0.6864

Epoch #16:
Train SN-UMG: 0.7544
Val SN-UMG: 0.6868

Epoch #17:
Train SN-UMG: 0.6487
Val SN-UMG: 0.6826

Epoch #18:
Train SN-UMG: 0.7072
Val SN-UMG: 0.6826

Epoch #19:
Train SN-UMG: 0.7466
Val SN-UMG: 0.6863

Epoch #20:
Train SN-UMG